In [23]:
import time
import datetime
import pytz
import requests
import json
from base64 import b64encode
import os
from dotenv import load_dotenv

In [8]:
# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variables
api_token_string = os.getenv('API_KEY')

# Check if the API key was loaded correctly
if api_token_string is None:
    raise ValueError("API_KEY environment variable not set")

api_token_string = api_token_string + ':api_token'    

In [7]:
TOGGL_API_URL = 'https://api.track.toggl.com/api/v9/me'

headers = {
            'content-type': 'application/json',
            'Authorization' : 'Basic %s' %  b64encode(api_token_string.encode('utf-8')).decode("ascii")
          }

In [ ]:
unix_time_minus_one_hour = int(time.time()) - 3600
print(unix_time_minus_one_hour)

In [ ]:
response = requests.get(TOGGL_API_URL + f'/time_entries?since={unix_time_minus_one_hour}', headers=headers)

# Ensure the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Pretty-print the JSON response
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
# Extract the 'start' field
start_time_str = data[0]['start']

# Convert to a datetime object
start_time_utc = datetime.datetime.fromisoformat(start_time_str)

# Convert to local time
local_timezone = pytz.timezone('America/New_York')  # Replace with your local timezone
start_time_local = start_time_utc.astimezone(local_timezone)

print(f"Start time in local time: {start_time_local}")

In [ ]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]


def cal_test():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")


cal_test()

In [ ]:
def get_credentials():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  return(creds)

creds = get_credentials()
creds

In [ ]:
event = {
  'summary': 'Summary - Test Add Event',
  'location': '13 Elm',
  'description': 'Description - Test Add Event',
  'start': {
    'dateTime': '2024-07-07T21:00:00-04:00',
    'timeZone': 'America/New_York',
  },
  'end': {
    'dateTime': '2024-07-07T22:00:00-04:00',
    'timeZone': 'America/New_York',
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;COUNT=2'
  ],
  'attendees': [
    {'email': 'lpage@example.com'},
    {'email': 'sbrin@example.com'},
  ],
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 10},
    ],
  },
}

try:
    service = build("calendar", "v3", credentials=creds)

    event = service.events().insert(calendarId='primary', body=event).execute()
    print ('Event created: %s' % (event.get('htmlLink')))

except HttpError as error:
    print(f"An error occurred: {error}")
